# 🚀 MARIA HELENA TRADING BOT - LSTM COMPLETO

## Treinamento de Modelo de Deep Learning para Previsão de Preços Bitcoin

**Desenvolvedor:** Marcos Sea (WSS13Framework)  
**Email:** wss13.framework@gmail.com  
**Empresa:** WSS+13  
**Data:** 2025-11-07  

---

### 📊 O que vamos fazer:
1. Carregar 721 candles históricos de Bitcoin
2. Calcular indicadores técnicos
3. Treinar modelo LSTM (Long Short-Term Memory)
4. Fazer predições de preço
5. Salvar modelo treinado

### ⏱️ Tempo estimado: 15-20 minutos
### 💻 GPU: Recomendado (gratuito no Colab)

## 1️⃣ INSTALAÇÃO E IMPORTS

Primeira etapa: instalar dependências e importar bibliotecas necessárias

In [ ]:
# Instalar bibliotecas
!pip install tensorflow scikit-learn pandas numpy matplotlib seaborn -q

print('⏳ Instalando bibliotecas...')
print('✅ Concluído!')

In [ ]:
# Importar bibliotecas
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configurações
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('✅ Bibliotecas importadas com sucesso!')
print(f'\n📊 Versões:')
print(f'   TensorFlow: {tf.__version__}')
print(f'   Pandas: {pd.__version__}')
print(f'   NumPy: {np.__version__}')

## 2️⃣ CARREGAR DADOS

Clone o repositório do GitHub e carregue os dados históricos de Bitcoin

In [ ]:
# Clone repositório
!git clone https://github.com/WSS13Framework/maria_helena_bot.git
%cd maria_helena_bot

print('✅ Repositório clonado!')

In [ ]:
# Carregar dados
df = pd.read_csv('bitcoin_training_data.csv')

print(f'✅ {len(df)} candles carregados!')
print(f'\n📋 Primeiras linhas:')
print(df.head(10))

print(f'\n📊 Informações do dataset:')
print(df.info())

print(f'\n📈 Estatísticas:')
print(df.describe())

In [ ]:
# Visualizar histórico de preços
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['close'], label='Close Price', linewidth=2, color='blue')
plt.title('📈 Histórico de Preços Bitcoin (721 candles)', fontsize=14, fontweight='bold')
plt.xlabel('Dias', fontsize=12)
plt.ylabel('Preço (USD)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'💰 Resumo de preços:')
print(f'   Preço mínimo: ${df["close"].min():,.2f}')
print(f'   Preço máximo: ${df["close"].max():,.2f}')
print(f'   Preço médio: ${df["close"].mean():,.2f}')
print(f'   Preço atual: ${df["close"].iloc[-1]:,.2f}')

## 3️⃣ PREPARAÇÃO DOS DADOS

Normalizar dados e criar sequências para o LSTM

In [ ]:
# Extrair preços de fechamento
data = df['close'].values.reshape(-1, 1)

print(f'📊 Dados originais:')
print(f'   Shape: {data.shape}')
print(f'   Min: ${data.min():,.2f}')
print(f'   Max: ${data.max():,.2f}')
print(f'   Média: ${data.mean():,.2f}')

# Normalizar entre 0 e 1 (importante para redes neurais)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

print(f'\n✅ Dados normalizados!')
print(f'   Min normalizado: {scaled_data.min():.4f}')
print(f'   Max normalizado: {scaled_data.max():.4f}')

In [ ]:
# Criar sequências para LSTM
# Usar 60 dias anteriores para prever o próximo dia
lookback = 60  # Janela de 60 dias

X_train = []
y_train = []

for i in range(lookback, len(scaled_data)):
    # Adicionar 60 dias anteriores como features
    X_train.append(scaled_data[i-lookback:i, 0])
    # Adicionar próximo dia como target
    y_train.append(scaled_data[i, 0])

X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshape para LSTM: [samples, timesteps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print(f'📈 Sequências criadas:')
print(f'   X_train shape: {X_train.shape}  (samples, timesteps, features)')
print(f'   y_train shape: {y_train.shape}  (targets)')
print(f'\n💡 Interpretação:')
print(f'   {X_train.shape[0]} amostras de treinamento')
print(f'   {X_train.shape[1]} timesteps (60 dias)')
print(f'   {X_train.shape[2]} feature (preço de fechamento)')

## 4️⃣ CONSTRUIR MODELO LSTM

Criar arquitetura de rede neural com 2 camadas LSTM

In [ ]:
# Criar modelo LSTM
model = Sequential([
    # Primeira camada LSTM com 50 unidades
    LSTM(50, activation='relu', input_shape=(lookback, 1), name='LSTM_1'),
    # Dropout para evitar overfitting (20%)
    Dropout(0.2, name='Dropout_1'),
    
    # Segunda camada LSTM com 50 unidades
    LSTM(50, activation='relu', name='LSTM_2'),
    # Dropout para evitar overfitting (20%)
    Dropout(0.2, name='Dropout_2'),
    
    # Camada densa com 25 unidades
    Dense(25, activation='relu', name='Dense_1'),
    
    # Camada de saída com 1 unidade (previsão de preço)
    Dense(1, name='Output')
])

# Compilar modelo
model.compile(
    optimizer='adam',      # Adam é um otimizador eficiente
    loss='mse',            # Mean Squared Error para regressão
    metrics=['mae']        # Mean Absolute Error como métrica
)

print('🧠 ARQUITETURA DO MODELO:')
print('=' * 70)
model.summary()
print('=' * 70)

print(f'\n📊 Resumo:')
print(f'   Total de parâmetros: {model.count_params():,}')
print(f'   Camadas: {len(model.layers)}')
print(f'   Loss: MSE (Mean Squared Error)')
print(f'   Optimizer: Adam')

## 5️⃣ TREINAR MODELO

Treinar a rede neural com os dados preparados (⏳ ~5-10 minutos)

In [ ]:
# Callback para parar o treinamento se não melhorar
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

print('🚀 INICIANDO TREINAMENTO DO MODELO...')
print('=' * 70)
print(f'Epochs: 50')
print(f'Batch size: 32')
print(f'Validation split: 20%')
print(f'Total de amostras de treino: {int(X_train.shape[0] * 0.8)}')
print(f'Total de amostras de validação: {int(X_train.shape[0] * 0.2)}')
print('=' * 70)

# Treinar
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stop]
)

print('\n✅ TREINAMENTO CONCLUÍDO COM SUCESSO!')

## 6️⃣ ANÁLISE DO TREINAMENTO

Visualizar loss e métricas de treinamento

In [ ]:
# Plotar Loss
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(history.history['loss'], label='Train Loss', linewidth=2.5, color='#2E86AB')
ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2.5, color='#A23B72')
ax1.set_title('Model Loss', fontsize=13, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=11)
ax1.set_ylabel('Loss (MSE)', fontsize=11)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# MAE
ax2.plot(history.history['mae'], label='Train MAE', linewidth=2.5, color='#2E86AB')
ax2.plot(history.history['val_mae'], label='Validation MAE', linewidth=2.5, color='#A23B72')
ax2.set_title('Model MAE', fontsize=13, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=11)
ax2.set_ylabel('MAE', fontsize=11)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'📊 RESUMO DO TREINAMENTO:')
print(f'   Loss inicial: {history.history["loss"][0]:.6f}')
print(f'   Loss final: {history.history["loss"][-1]:.6f}')
print(f'   Val Loss inicial: {history.history["val_loss"][0]:.6f}')
print(f'   Val Loss final: {history.history["val_loss"][-1]:.6f}')
print(f'   Epochs executados: {len(history.history["loss"])}')

## 7️⃣ FAZER PREDIÇÕES

Avaliar modelo nos dados de treinamento

In [ ]:
# Predições nos dados de treinamento
print('🔮 Fazendo predições nos dados de treinamento...')
train_predict = model.predict(X_train, verbose=0)

# Reverter normalização
train_predict = scaler.inverse_transform(train_predict)
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))

# Calcular métricas
train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predict))
train_mae = mean_absolute_error(y_train_actual, train_predict)
train_r2 = r2_score(y_train_actual, train_predict)

print(f'\n📈 MÉTRICAS DO MODELO:')
print(f'   RMSE: ${train_rmse:,.2f}')
print(f'   MAE: ${train_mae:,.2f}')
print(f'   R² Score: {train_r2:.4f}')

In [ ]:
# Visualizar predições vs real
plt.figure(figsize=(14, 6))
plt.plot(y_train_actual, label='Preço Real', linewidth=2.5, color='#2E86AB', alpha=0.8)
plt.plot(train_predict, label='Preço Predito', linewidth=2.5, color='#A23B72', alpha=0.8)
plt.title('Maria Helena LSTM - Predições vs Real (Dados de Treinamento)', fontsize=14, fontweight='bold')
plt.xlabel('Dias', fontsize=12)
plt.ylabel('Preço (USD)', fontsize=12)
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('✅ Gráfico exibido!')

## 8️⃣ PREVER PRÓXIMO PREÇO 🎯

Usar os últimos 60 candles para prever o próximo preço

In [ ]:
# Pegar últimos 60 candles
last_60_days = data[-60:]
last_60_days_scaled = scaler.transform(last_60_days)

# Preparar para predição
X_test = np.array([last_60_days_scaled])
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Fazer predição
next_price_scaled = model.predict(X_test, verbose=0)
next_price = scaler.inverse_transform(next_price_scaled)

# Calcular mudança
current_price = data[-1][0]
change = next_price[0][0] - current_price
change_pct = (change / current_price) * 100

print('🚀 PREDIÇÃO DO PRÓXIMO PREÇO')
print('=' * 70)
print(f'💰 Preço atual: ${current_price:,.2f}')
print(f'📈 Preço predito: ${next_price[0][0]:,.2f}')
print(f'📊 Mudança: ${change:,.2f} ({change_pct:+.2f}%)')
print('=' * 70)

# Determinar signal
if change_pct > 2:
    signal = '🟢 BUY (Preço pode subir >2%)'
    confidence = min(abs(change_pct) / 10, 1.0)
elif change_pct < -2:
    signal = '🔴 SELL (Preço pode cair >2%)'
    confidence = min(abs(change_pct) / 10, 1.0)
else:
    signal = '🟡 HOLD (Mudança < 2%)'
    confidence = 0.5

print(f'\n⚡ SIGNAL: {signal}')
print(f'📊 Confiança: {confidence*100:.1f}%')

## 9️⃣ SALVAR MODELO

Salvar modelo treinado para uso posterior

In [ ]:
# Salvar modelo
model.save('maria_helena_lstm_model.h5')

import os
file_size = os.path.getsize('maria_helena_lstm_model.h5') / (1024 * 1024)

print('✅ MODELO SALVO COM SUCESSO!')
print(f'   Arquivo: maria_helena_lstm_model.h5')
print(f'   Tamanho: {file_size:.2f} MB')
print(f'   Local: {os.path.abspath("maria_helena_lstm_model.h5")}')

## 🔟 DOWNLOAD E PRÓXIMOS PASSOS

Baixar modelo e fazer upload no servidor

In [ ]:
# Download do modelo
from google.colab import files

print('📥 BAIXANDO MODELO...')
print('=' * 70)
files.download('maria_helena_lstm_model.h5')
print('\n✅ DOWNLOAD INICIADO!')
print('   Verifique a pasta Downloads')

In [ ]:
print('🚀 PRÓXIMOS PASSOS')
print('=' * 70)
print('\n1️⃣ SALVAR O MODELO')
print('   Arquivo: maria_helena_lstm_model.h5')
print('   Local: Downloads do seu computador')

print('\n2️⃣ FAZER UPLOAD PRO SERVIDOR')
print('   Comando: scp maria_helena_lstm_model.h5 root@seu-server:/root/maria-helena-scripts/')

print('\n3️⃣ RODAR PREDIÇÕES AUTOMÁTICAS')
print('   Comando: python3 /root/maria-helena-scripts/run_lstm_predictions.py')

print('\n4️⃣ IMPORTAR WORKFLOWS N8N')
print('   Dashboard: http://seu-server:5678')
print('   Importar: 4 JSONs dos workflows')

print('\n5️⃣ ATIVAR TRADES AUTOMÁTICOS')
print('   Configurar: API keys da exchange')
print('   Money management: 1% risco por trade')

print('\n' + '=' * 70)
print('✅ MARIA HELENA TRADING BOT - MODELO TREINADO COM SUCESSO!')
print('=' * 70)

print('\n📞 CONTATO:')
print('   Desenvolvedor: Marcos Sea')
print('   Email: wss13.framework@gmail.com')
print('   GitHub: github.com/WSS13Framework')
print('   Empresa: WSS+13')